<a href="https://colab.research.google.com/github/mahdieslaminet/ML-Final-Project/blob/main/Futsal_Talent_AI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# =========================
# ماژول‌ها
# =========================
import numpy as np
import cv2
import tensorflow as tf
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Flatten, Concatenate
from tensorflow.keras.optimizers import Adam
from sklearn.preprocessing import MinMaxScaler
from google.colab import files
from io import BytesIO

# =========================
# استانداردهای فوتسال
# =========================
MIN_HEIGHT = 1.60
MAX_HEIGHT = 2.00
MIN_BMI = 18
MAX_BMI = 30
MAX_PRO_START_AGE = 30
MIN_SKILL = 5
MIN_AVG_SKILL = 6

positions = {
    "Goalkeeper": np.array([1.85, 85, 6, 6, 7, 7, 6, 6]),
    "Defender":   np.array([1.78, 75, 7, 6, 7, 7, 6, 5]),
    "Midfielder": np.array([1.75, 70, 8, 8, 8, 8, 8, 6]),
    "Winger":     np.array([1.72, 68, 9, 9, 7, 7, 9, 7]),
    "Forward":    np.array([1.80, 78, 8, 7, 7, 6, 8, 9]),
    "Pivot":      np.array([1.85, 85, 7, 6, 8, 7, 6, 9]),
}

# =========================
# تابع پردازش تصویر
# =========================
def process_image_file(file_bytes, target_size=(224,224)):
    arr = np.frombuffer(file_bytes, np.uint8)
    img = cv2.imdecode(arr, cv2.IMREAD_COLOR)
    img_resized = cv2.resize(img, target_size)
    img_rgb = cv2.cvtColor(img_resized, cv2.COLOR_BGR2RGB)
    img_normalized = img_rgb / 255.0
    img_batch = np.expand_dims(img_normalized, axis=0)
    return img_batch

# =========================
# آپلود تصویر
# =========================
print("لطفاً تصویر بازیکن را آپلود کنید:")
uploaded = files.upload()
img_name = list(uploaded.keys())[0]
img_bytes = uploaded[img_name]
img_ready = process_image_file(img_bytes)
print("تصویر آماده و پردازش شد!")

# =========================
# ورود داده متنی بازیکن
# =========================
print("=== Enter New Player Information ===")
height = float(input("Height (m or cm): "))
weight = float(input("Weight (kg): "))
start_age = int(input("Professional start age: "))
speed = int(input("Speed (1-10): "))
agility = int(input("Agility (1-10): "))
endurance = int(input("Endurance (1-10): "))
passing = int(input("Passing skill (1-10): "))
dribbling = int(input("Dribbling skill (1-10): "))
shooting = int(input("Shooting skill (1-10): "))

if height > 3:
    height /= 100
    print(f"⚠️ Height converted to meters: {height:.2f} m")

# =========================
# استخراج ویژگی تصویر با MobileNetV2
# =========================
base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(224,224,3))
x = Flatten()(base_model.output)
image_model = Model(inputs=base_model.input, outputs=x)
image_features = image_model.predict(img_ready)
print(f"ویژگی تصویر استخراج شد: {image_features.shape}")

# =========================
# مدل ترکیبی تصویر + داده متنی
# =========================
text_features = np.array([[height, weight, start_age, speed, agility, endurance, passing, dribbling, shooting]])
scaler = MinMaxScaler()
text_features_scaled = scaler.fit_transform(text_features)

input_text = Input(shape=(text_features_scaled.shape[1],))
input_image = Input(shape=(image_features.shape[1],))
combined = Concatenate()([input_text, input_image])
x = Dense(128, activation='relu')(combined)
x = Dense(64, activation='relu')(x)
output = Dense(len(positions), activation='softmax')(x)
final_model = Model(inputs=[input_text, input_image], outputs=output)
final_model.compile(optimizer=Adam(0.001), loss='categorical_crossentropy', metrics=['accuracy'])

# برای نمونه از همین داده کوچک آموزش می‌دهیم
X_image_features = np.repeat(image_features, len(positions), axis=0)
X_text_scaled = np.repeat(text_features_scaled, len(positions), axis=0)
y_dummy = np.eye(len(positions))
final_model.fit([X_text_scaled, X_image_features], y_dummy, epochs=3, batch_size=4, verbose=0)

# پیش‌بینی پست
pred = final_model.predict([text_features_scaled, image_features])
pred_class = list(positions.keys())[np.argmax(pred)]
print(f"\nپیش‌بینی مدل ترکیبی: {pred_class}")

# =========================
# ارزیابی استاندارد فوتسال
# =========================
bmi = weight / (height ** 2)
rejection_reasons = []

if height < MIN_HEIGHT:
    rejection_reasons.append("قد کوتاه‌تر از استاندارد فوتسال")
if bmi > MAX_BMI:
    rejection_reasons.append("وزن بالا نسبت به قد (BMI بالا)")
if bmi < MIN_BMI:
    rejection_reasons.append("وزن پایین نسبت به قد (BMI پایین)")
if start_age > MAX_PRO_START_AGE:
    rejection_reasons.append("سن شروع حرفه‌ای دیرتر از بازه مناسب")

skills = [speed, agility, endurance, passing, dribbling, shooting]
weak_skills = [name for i,name in zip(skills, ["سرعت","چابکی","استقامت","پاس","دریبل","شوت"]) if i < MIN_SKILL]
if len(weak_skills) >= 2:
    rejection_reasons.append("ضعف در مهارت‌های کلیدی فوتسال: " + "، ".join(weak_skills))

if np.mean(skills) < MIN_AVG_SKILL:
    rejection_reasons.append("میانگین کلی مهارت‌ها پایین‌تر از حد استاندارد فوتسال")

# =========================
# نتیجه نهایی
# =========================
print("\n===============================")
print("📊 FINAL ANALYSIS RESULT")
print("===============================")

if rejection_reasons:
    print("❌ Suitability: Not suitable for futsal\n")
    print("📌 Reasons for rejection:")
    for r in rejection_reasons:
        print(f" - {r}")
else:
    print("✅ Suitability: Suitable for futsal\n")
    print(f"🏷️ Suggested Position (from model): {pred_class}")

لطفاً تصویر بازیکن را آپلود کنید:


Saving images (4).jpeg to images (4).jpeg
تصویر آماده و پردازش شد!
=== Enter New Player Information ===
Height (m or cm): 180
Weight (kg): 75
Professional start age: 20
Speed (1-10): 9
Agility (1-10): 9
Endurance (1-10): 8
Passing skill (1-10): 9
Dribbling skill (1-10): 9
Shooting skill (1-10): 9
⚠️ Height converted to meters: 1.80 m
9406464/9406464 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
ویژگی تصویر استخراج شد: (1, 62720)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 76ms/step

پیش‌بینی مدل ترکیبی: Goalkeeper

📊 FINAL ANALYSIS RESULT
✅ Suitability: Suitable for futsal

🏷️ Suggested Position (from model): Goalkeeper
